<a href="https://colab.research.google.com/github/niel-blue/niel-blue-beatrice-trainer-webui-colab/blob/main/Beatrice_Trainer_unoffisial_webui_colab_0113.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. beatrice-trainer インストールと環境構築
from google.colab import drive
import os

# Driveのマウント
drive.mount('/content/drive')

# リポジトリのクローン
!git clone https://huggingface.co/fierce-cats/beatrice-trainer
%cd /content/beatrice-trainer

# 依存関係のインストール (uvは使わずpipで直接入れます)
!pip install -e .[cu128]
!pip install gradio pydub

# 必要なシステムパッケージ
!apt-get install -y ffmpeg

print("インストールが完了しました。")

In [ ]:
# @title 2. WebUIファイルの生成 (webui_colab.py)
code = r'''
import gradio as gr
import json
import os
import subprocess
import webbrowser
import locale
import importlib
import shutil
import torch
import torchaudio
from pathlib import Path
import multiprocessing
import threading
import time
import queue
import re
from pydub import AudioSegment, silence

# グローバル変数など
# torchaudio.set_audio_backend("sox_io") # 警告が出るため削除

training_process = None
is_terminated_by_user = False
training_tasks = []
current_task = None
training_thread = None
current_dir = os.getcwd()
default_config_path = os.path.join(current_dir, "assets", "default_config.json")
is_slicing_terminated_by_user = False

# --- Colab/Drive用の定数定義 ---
DRIVE_ROOT = "/content/drive/MyDrive"
LOCAL_TEMP_ROOT = "/content/temp_dataset"

os.makedirs(LOCAL_TEMP_ROOT, exist_ok=True)

# 文言のハードコード定義
locale_data = {
    "LNG_TITLE": "Beatrice-Trainer Unofficial Simple WebUI (for 2.0.0-rc.0) - Colab Edition",
    "LNG_TRAIN_DESC": "各種を設定し[タスクをキューに登録]ボタンでタスクを登録（※複数可）、[トレーニング開始]でタスクが順番に実行される。",
    "LNG_STOP_SUCCESS_MESSAGE": "トレーニングプロセスを停止しました。キューをクリアします。",
    "LNG_STOP_NO_PROCESS_MESSAGE": "実行中のトレーニングプロセスはありません。キューをクリアします。",
    "LNG_RESUME_WARNING": "指定されたn_stepsの値 ({n_steps_val}) が、前回保存されたconfig.jsonのステップ数 ({existing_steps}) より少ないです。n_stepsの値が正しいか確認してください。",
    "LNG_CONFIG_LOAD_ERROR": "config.jsonの読み込み中に問題が発生しました: {e}",
    "LNG_CONFIG_NOT_FOUND_WARNING": "追加学習用のconfig.jsonが見つかりませんでした。新規学習として扱います。",
    "LNG_LATEST_CHECKPOINT_BACKUP": "既存の最新チェックポイントをバックアップしました: {backup_path}",
    "LNG_NO_AUDIO_FILES_IN_ROOT_ALERT": "指定されたフォルダの直下に音声ファイルがあります。話者ごとにサブフォルダを作成し、その中に音声を配置してください。",
    "LNG_NO_AUDIO_FILES_IN_SUBFOLDERS_ALERT": "音声ファイルが見つかりません。話者フォルダ内に音声を配置しているか確認してください。",
    "LNG_INPUT_FOLDER": "[データフォルダの指定]",
    "LNG_INPUT_FOLDER_INFO": "マイドライブ/データセットフォルダ/話者名フォルダ/音声ファイル　という配置にしてください。なお、データセットのフォルダ名がモデル名となります。",
    "LNG_INPUT_FOLDER_PLACE": "例: マイドライブ/dataset1/話者名1/XXX.wav　の場合、dataset1 とだけ記述。",
    "LNG_INPUT_FOLDER_ALERT": "データフォルダのパスが入力されていないか、Drive上に存在しません",
    "LNG_OUTPUT_FOLDER": "[出力先、および追加学習フォルダの指定]",
    "LNG_OUTPUT_FOLDER_INFO": "新規の場合は必ず空のフォルダを指定してください。前の学習モデルがある場合は自動的に追加学習扱いになります。",
    "LNG_OUTPUT_FOLDER_PLACE":  "例: マイドライブ/output に出力、もしくは途中までの学習ファイルがある場合、output とだけ記述。",
    "LNG_OUTPUT_FOLDER_ALERT": "出力先フォルダのパスが入力されていないか、パスが間違っています",
    "LNG_CONFIG_SAVE_INFO": "config.json が生成されました。",
    "LNG_BASIC_TRAINING": "基本トレーニング設定",
    "LNG_N_STEPS_INFO": "トレーニングの総ステップ数。品質にも大きく影響。増やすと学習時間も増加する。",
    "LNG_BATCH_SIZE_INFO": "一度に処理する学習データ数。増やすと安定性が向上するなど学習速度や品質に大きく影響するがVRAM使用量も増加。",
    "LNG_NUM_WORKERS_INFO": "データ読み込みを並列処理するスレッド数。CPUコア数に合わせると効率が上がりますがメモリ消費も増加。",
    "LNG_SAVE_INTERVAL_INFO": "指定ステップごとにチェックポイントファイルを保存します。colab無料版は切断リスク回避のために1000～2000ぐらいに設定すると無難。",
    "LNG_EVALUATION_INTERVAL_INFO": "指定ステップごとにテスト音声の生成を行い、学習の進み具合を検証。",
    "LNG_ADVANCED_OPTIONS": "詳細設定",
    "LNG_LEARNING_RATE_OPTIMIZER": "学習率・オプティマイザ",
    "LNG_LOSS_WEIGHTS": "損失関数の重み",
    "LNG_AUGMENTATION_OPTIONS": "データ拡張オプション",
    "LNG_AUDIO_MODEL": "オーディオ・モデルパラメータ",
    "LNG_FILE_PATHS": "ファイルパス",
    "LNG_PERFORMANCE_DEBUG": "パフォーマンス・デバッグ",
    "LNG_IN_SAMPLE_RATE_INFO": "変更不可。",
    "LNG_OUT_SAMPLE_RATE_INFO": "変更不可。",
    "LNG_TASK_MONITOR_EMPTY": "現在キューは空です。タスクを追加してください。",
    "LNG_QUEUE_TASK_RESUMED": "追加学習としてキューに登録しました: Output={output_folder}",
    "LNG_QUEUE_TASK_NEW": "新規学習としてキューに登録しました: Output={output_folder}",
    "LNG_QUEUE_STATUS_PENDING": "[待機中◼]",
    "LNG_QUEUE_STATUS_IN_PROGRESS": "[実行中⏩]",
    "LNG_QUEUE_STATUS_COMPLETED": "[完了済✅]",
    "LNG_QUEUE_STATUS_ERROR": "[エラー❌]",
    "LNG_QUEUE_STATUS_STOPPED": "[中止🛑]",
    "LNG_QUEUE_START_INFO": "前処理と学習を開始します... Input={local_in}, Output={drive_out}",
    "LNG_USER_TERMINATED_MESSAGE": "タスクはユーザーにより中断されました。",
    "LNG_ALL_TASKS_COMPLETED_MESSAGE": "キュー内の全タスクが完了しました。",
    "LNG_ADD_TASK_BUTTON": "タスクをキューに登録",
    "LNG_START_TRAINING_BUTTON": "トレーニング開始",
    "LNG_STOP_TRAINING_BUTTON": "中断/キューのクリア",
    "LNG_AUDIO_PREPROCESS_SETTINGS": "音声ファイル変換設定",
    "LNG_ENABLE_PREPROCESS": "音声ファイル変換を行う",
    "LNG_ENABLE_PREPROCESS_INFO": "モノラル 16bitPCM変換、無音部分削除や分割など、事前処理した音声データをサーバに送ります。事前に処理した音声ファイルを使用する場合はチェックを外してください。",
    "LNG_SEGMENT_DURATION_LABEL": "音声の分割長さ（秒）",
    "LNG_SEGMENT_DURATION_INFO": "音声を指定した長さで分割します。（※5秒程度 推奨）",
    "LNG_OUTPUT_SAMPLERATE_LABEL": "サンプリングレート",
    "LNG_OUTPUT_SAMPLERATE_INFO": "出力する音声のサンプリングレートを選択（※16000Hz 推奨）",
    "LNG_OUTPUT_FORMAT_LABEL": "出力形式を選択",
    "LNG_OUTPUT_FORMAT_INFO": "保存するファイル形式を選択してください。",
    "LNG_SILENCE_REMOVAL_OPTIONS": "無音部分削除オプション",
    "LNG_SILENCE_REMOVAL_OPTIONS_INFO": "無音部分を自動で除去するかどうかを制御します（推奨）",
    "LNG_SILENCE_THRESHOLD_LABEL": "無音の閾値（dBFS）",
    "LNG_SILENCE_THRESHOLD_INFO": "この値より小さい音量を無音と見なします。",
    "LNG_MIN_SILENCE_DURATION_LABEL": "最小無音時間（ミリ秒）",
    "LNG_MIN_SILENCE_DURATION_INFO": "この時間よりも長い無音部分を除去します。"
}

# デフォルトconfig.jsonの読み込み
with open(default_config_path, "r", encoding="utf-8") as f:
    default_config = json.load(f)

# --- 音声前処理関数 ---
def preprocess_dataset(drive_input_path, local_output_path, prep_params):

    (enable_prep, duration_sec, sample_rate, out_format,
     enable_silence, silence_thresh, min_silence_ms) = prep_params

    print(f"Pre-processing started: {drive_input_path} -> {local_output_path}")

    if os.path.exists(local_output_path):
        shutil.rmtree(local_output_path)
    os.makedirs(local_output_path, exist_ok=True)

    audio_extensions = {".wav", ".flac", ".ogg", ".aiff", ".mp3", ".m4a"}
    input_path_obj = Path(drive_input_path)
    total_files = 0

    if not enable_prep:
        print("Processing disabled. Copying files directly...")
        for root, dirs, files in os.walk(drive_input_path):
            for file in files:
                if Path(file).suffix.lower() in audio_extensions:
                    src_file = Path(root) / file
                    rel_path = src_file.relative_to(input_path_obj)
                    dest_file = Path(local_output_path) / rel_path
                    dest_file.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(src_file, dest_file)
                    total_files += 1
        print(f"Copy finished. Total {total_files} files copied.")
        return True

    # 処理を行う場合 (Colabオリジナルのpydubロジックを使用)
    target_len_ms = int(duration_sec * 1000)

    for root, dirs, files in os.walk(drive_input_path):
        for file in files:
            if Path(file).suffix.lower() in audio_extensions:
                file_path = Path(root) / file
                rel_path = file_path.relative_to(input_path_obj)
                parts = rel_path.parts
                if len(parts) < 2:
                    continue

                speaker_name = parts[0]
                speaker_dir_local = os.path.join(local_output_path, speaker_name)
                os.makedirs(speaker_dir_local, exist_ok=True)

                try:
                    # pydubによる読み込み
                    audio = AudioSegment.from_file(file_path)

                    # パラメータ適用: モノラル化、16bit化を同時に実行
                    audio = audio.set_frame_rate(int(sample_rate)).set_channels(1).set_sample_width(2)

                    # パラメータ適用: 無音除去
                    if enable_silence:
                        chunks = silence.split_on_silence(
                            audio,
                            min_silence_len=int(min_silence_ms),
                            silence_thresh=int(silence_thresh)
                        )
                    else:
                        chunks = [audio] # 無音除去しない場合は全体を処理

                    if not chunks:
                        chunks = [audio]

                    processed_chunks = []
                    for chunk in chunks:
                        if len(chunk) < 500:
                            continue
                        # パラメータ適用: 分割長さ
                        for i in range(0, len(chunk), target_len_ms):
                            slice_chunk = chunk[i : i + target_len_ms]
                            if len(slice_chunk) > 500:
                                processed_chunks.append(slice_chunk)

                    base_name = file_path.stem
                    for idx, pc in enumerate(processed_chunks):
                        # パラメータ適用: 出力形式
                        out_name = f"{base_name}_{idx:03d}.{out_format}"
                        pc.export(os.path.join(speaker_dir_local, out_name), format=out_format)
                        total_files += 1
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

    print(f"Pre-processing finished. Total {total_files} files generated.")
    return True

# IOフォルダのパスチェック関数
def path_check(input_folder_name, output_folder_name):
    drive_in = os.path.join(DRIVE_ROOT, input_folder_name.strip())
    drive_out = os.path.join(DRIVE_ROOT, output_folder_name.strip())
    if not input_folder_name or not os.path.exists(drive_in):
        gr.Warning(locale_data["LNG_INPUT_FOLDER_ALERT"] + f"\\nExpected: {drive_in}")
        return False, drive_in, drive_out
    if not output_folder_name:
        gr.Warning(locale_data["LNG_OUTPUT_FOLDER_ALERT"])
        return False, drive_in, drive_out
    return True

# 音声ファイルチェック関数
def has_audio_files_recursive(input_folder_path):
    audio_extensions = {".wav", ".flac", ".ogg", ".aiff", ".mp3"}
    input_path = Path(input_folder_path)
    for root, dirs, files in os.walk(input_path):
        for file in files:
            if Path(file).suffix.lower() in audio_extensions:
                rel = Path(root).relative_to(input_path)
                if str(rel) != ".":
                    return True
    return False

def str_to_float_list(s):
    if not isinstance(s, str):
        return s
    return [float(item.strip()) for item in s.split(',') if item.strip()]

# 設定ファイル生成用関数
def generate_config(
    learning_rate_g, learning_rate_d, learning_rate_decay, adam_betas_1, adam_betas_2, adam_eps,
    batch_size, grad_weight_loudness, grad_weight_mel, grad_weight_ap, grad_weight_adv,
    grad_weight_fm, grad_balancer_ema_decay, use_amp, num_workers, n_steps, warmup_steps,
    evaluation_interval, save_interval,
    in_sample_rate, out_sample_rate, wav_length, segment_length, phone_noise_ratio, vq_topk,
    training_time_vq, floor_noise_level, record_metrics,
    aug_snr_candidates, aug_formant_shift_prob, aug_formant_shift_min, aug_formant_shift_max,
    aug_reverb_prob, aug_lpf_prob, aug_lpf_cutoff_candidates,
    phone_extractor_file, pitch_estimator_file, in_ir_wav_dir, in_noise_wav_dir,
    in_test_wav_dir, pretrained_file,
    pitch_bins, hidden_channels, san, compile_convnext, compile_d4c, compile_discriminator, profile,
    output_folder_path
):
    config = {
        "learning_rate_g": float(learning_rate_g),
        "learning_rate_d": float(learning_rate_d),
        "learning_rate_decay": float(learning_rate_decay),
        "adam_betas": [float(adam_betas_1), float(adam_betas_2)],
        "adam_eps": float(adam_eps),
        "batch_size": int(batch_size),
        "grad_weight_loudness": float(grad_weight_loudness),
        "grad_weight_mel": float(grad_weight_mel),
        "grad_weight_ap": float(grad_weight_ap),
        "grad_weight_adv": float(grad_weight_adv),
        "grad_weight_fm": float(grad_weight_fm),
        "grad_balancer_ema_decay": float(grad_balancer_ema_decay),
        "use_amp": bool(use_amp),
        "num_workers": int(num_workers),
        "n_steps": int(n_steps),
        "warmup_steps": int(warmup_steps),
        "evaluation_interval": int(evaluation_interval),
        "save_interval": int(save_interval),
        "in_sample_rate": int(in_sample_rate),
        "out_sample_rate": int(out_sample_rate),
        "wav_length": int(wav_length),
        "segment_length": int(segment_length),
        "phone_noise_ratio": float(phone_noise_ratio),
        "vq_topk": int(vq_topk),
        "training_time_vq": str(training_time_vq),
        "floor_noise_level": float(floor_noise_level),
        "record_metrics": bool(record_metrics),
        "augmentation_snr_candidates": str_to_float_list(aug_snr_candidates),
        "augmentation_formant_shift_probability": float(aug_formant_shift_prob),
        "augmentation_formant_shift_semitone_min": float(aug_formant_shift_min),
        "augmentation_formant_shift_semitone_max": float(aug_formant_shift_max),
        "augmentation_reverb_probability": float(aug_reverb_prob),
        "augmentation_lpf_probability": float(aug_lpf_prob),
        "augmentation_lpf_cutoff_freq_candidates": str_to_float_list(aug_lpf_cutoff_candidates),
        "phone_extractor_file": phone_extractor_file,
        "pitch_estimator_file": pitch_estimator_file,
        "in_ir_wav_dir": in_ir_wav_dir,
        "in_noise_wav_dir": in_noise_wav_dir,
        "in_test_wav_dir": in_test_wav_dir,
        "pretrained_file": pretrained_file,
        "pitch_bins": int(pitch_bins),
        "hidden_channels": int(hidden_channels),
        "san": bool(san),
        "compile_convnext": bool(compile_convnext),
        "compile_d4c": bool(compile_d4c),
        "compile_discriminator": bool(compile_discriminator),
        "profile": bool(profile)
    }
    os.makedirs(output_folder_path, exist_ok=True)
    config_path = os.path.join(output_folder_path, "config.json")
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=4, ensure_ascii=False)
        gr.Info(locale_data["LNG_CONFIG_SAVE_INFO"])
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
    return config_path

def start_training_command(local_input_folder, drive_output_folder, is_resume):
    global training_process
    config_path = os.path.join(drive_output_folder, "config.json")
    command = [
        "python",
        "beatrice_trainer/__main__.py",
        "-d", local_input_folder,
        "-o", drive_output_folder,
        "-c", config_path
    ]
    if is_resume:
        command.append("-r")
    print(f"Executing command: {' '.join(command)}")
    training_process = subprocess.Popen(command)

def process_training_queue_generator():
    global training_process, is_terminated_by_user, training_tasks
    if not training_tasks:
        yield gr.Button(interactive=True), gr.Button(interactive=False), gr.Button(interactive=False), gr.Markdown(display_queue())
        return
    yield gr.Button(interactive=False), gr.Button(interactive=False), gr.Button(interactive=True), gr.Markdown(display_queue())
    for task in training_tasks:
        if is_terminated_by_user:
            break
        if task['status'] == 'pending':
            task['status'] = 'in_progress'
            yield gr.Button(interactive=False), gr.Button(interactive=False), gr.Button(interactive=True), gr.Markdown(display_queue())
            drive_input = task['drive_input']
            drive_output = task['drive_output']
            local_input = task['local_input']
            is_resume = task['is_resume']
            config_params = task['config_params']

            prep_params = task['prep_params']

            gr.Info(f"Preprocessing data from Drive to Colab Local Storage...")

            preprocess_success = preprocess_dataset(drive_input, local_input, prep_params)

            if not preprocess_success:
                 task['status'] = 'error'
                 continue
            gr.Info(locale_data["LNG_QUEUE_START_INFO"].format(local_in=local_input, drive_out=drive_output))
            generate_config(*config_params, output_folder_path=drive_output)
            start_training_command(local_input, drive_output, is_resume)
            while training_process.poll() is None:
                yield gr.Button(interactive=False), gr.Button(interactive=False), gr.Button(interactive=True), gr.Markdown(display_queue())
                time.sleep(1)
            if not is_terminated_by_user and training_process.poll() == 0:
                task['status'] = 'completed'
            elif is_terminated_by_user:
                task['status'] = 'stopped'
                break
            else:
                task['status'] = 'error'
                break
    training_process = None
    is_terminated_by_user = False
    yield gr.Button(interactive=True), gr.Button(interactive=True), gr.Button(interactive=False), gr.Markdown(display_queue())

def stop_training():
    global training_process, is_terminated_by_user, training_tasks
    if training_process and training_process.poll() is None:
        training_process.kill()
        gr.Info(locale_data["LNG_STOP_SUCCESS_MESSAGE"])
    else:
        gr.Info(locale_data["LNG_STOP_NO_PROCESS_MESSAGE"])
    training_tasks.clear()
    is_terminated_by_user = False
    return gr.Button(interactive=True), gr.Button(interactive=True), gr.Button(interactive=False), gr.Markdown(display_queue())

# キュー追加関数
def add_to_queue(input_folder_name, output_folder_name,
                 enable_prep, prep_duration, prep_sr, prep_format, enable_silence, silence_thresh, min_silence,
                 *args):
    global training_tasks

    # 1. Google Drive 上のパスを構築
    drive_in = os.path.join(DRIVE_ROOT, input_folder_name.strip())
    drive_out = os.path.join(DRIVE_ROOT, output_folder_name.strip())

    # 2. Google Drive 上に指定のフォルダがあるかを確認
    if not input_folder_name or not os.path.exists(drive_in):
        gr.Warning(locale_data["LNG_INPUT_FOLDER_ALERT"] + f"\nPath: {drive_in}")
        return gr.Markdown(display_queue()), gr.Button(interactive=True), gr.Button(interactive=False), gr.Button(interactive=False)

    # 3. Drive上の話者フォルダ内に音声があるかを確認
    if not has_audio_files_recursive(drive_in):
        gr.Warning(locale_data["LNG_NO_AUDIO_FILES_IN_SUBFOLDERS_ALERT"] + f"\nChecked: {drive_in}")
        return gr.Markdown(display_queue()), gr.Button(interactive=True), gr.Button(interactive=False), gr.Button(interactive=False)

    # 出力先パスのチェック
    if not output_folder_name:
        gr.Warning(locale_data["LNG_OUTPUT_FOLDER_ALERT"])
        return gr.Markdown(display_queue()), gr.Button(interactive=True), gr.Button(interactive=False), gr.Button(interactive=False)

    # ローカル（一時領域）のパスを定義する（まだ存在しなくて良い）
    local_in = os.path.join(LOCAL_TEMP_ROOT, os.path.basename(drive_in))

    # 追加学習かどうかを判断 (checkpoint_latest.pt.gzの有無で自動判定)
    is_resume = False
    latest_checkpoint_path = os.path.join(drive_out, "checkpoint_latest.pt.gz")
    if os.path.isfile(latest_checkpoint_path):
        is_resume = True

    if is_resume:
        config_path = os.path.join(drive_out, "config.json")
        if os.path.isfile(config_path):
            try:
                with open(config_path, "r", encoding="utf-8") as f:
                    existing_config = json.load(f)
                existing_steps = existing_config.get("n_steps", 0)
                n_steps_val = args[15]
                if n_steps_val <= existing_steps:
                    gr.Warning(locale_data["LNG_RESUME_WARNING"].format(n_steps_val=n_steps_val, existing_steps=existing_steps))
            except Exception as e:
                gr.Warning(locale_data["LNG_CONFIG_LOAD_ERROR"].format(e=e))
        else:
            gr.Warning(locale_data["LNG_CONFIG_NOT_FOUND_WARNING"])
            is_resume = False

    prep_params_data = (enable_prep, prep_duration, prep_sr, prep_format, enable_silence, silence_thresh, min_silence)

    task_data = {
        'drive_input': drive_in,
        'drive_output': drive_out,
        'local_input': local_in,
        'is_resume': is_resume,
        'prep_params': prep_params_data, # 保存
        'config_params': args,
        'status': 'pending'
    }
    training_tasks.append(task_data)
    if is_resume:
        gr.Info(locale_data["LNG_QUEUE_TASK_RESUMED"].format(output_folder=output_folder_name))
    else:
        gr.Info(locale_data["LNG_QUEUE_TASK_NEW"].format(output_folder=output_folder_name))
    return gr.Markdown(display_queue()), gr.Button(interactive=True), gr.Button(interactive=True), gr.Button(interactive=True)

def display_queue():
    global training_tasks
    if not training_tasks:
        return locale_data["LNG_TASK_MONITOR_EMPTY"]
    display_text = ""
    for i, task in enumerate(training_tasks):
        status_text = ""
        if task['status'] == 'pending': status_text = locale_data["LNG_QUEUE_STATUS_PENDING"]
        elif task['status'] == 'in_progress': status_text = locale_data["LNG_QUEUE_STATUS_IN_PROGRESS"]
        elif task['status'] == 'completed': status_text = locale_data["LNG_QUEUE_STATUS_COMPLETED"]
        elif task['status'] == 'error': status_text = locale_data["LNG_QUEUE_STATUS_ERROR"]
        elif task['status'] == 'stopped': status_text = locale_data["LNG_QUEUE_STATUS_STOPPED"]
        out_folder_name = os.path.basename(task['drive_output'])
        display_text += f"{i+1:02d} | {status_text} | Output:`{out_folder_name}` | n_steps: `{task['config_params'][15]}` | batch_size: `{task['config_params'][6]}`  \\n"
    return display_text

# --- UI構築 ---
with gr.Blocks() as demo:
    gr.HTML(f"<h1>{locale_data['LNG_TITLE']}</h1>")

    gr.Markdown(locale_data["LNG_TRAIN_DESC"])
    with gr.Row():
        with gr.Column():
            input_folder = gr.Textbox(
                label=locale_data["LNG_INPUT_FOLDER"],
                placeholder=locale_data["LNG_INPUT_FOLDER_PLACE"],
                info=locale_data["LNG_INPUT_FOLDER_INFO"]
            )
            output_folder = gr.Textbox(
                label=locale_data["LNG_OUTPUT_FOLDER"],
                placeholder=locale_data["LNG_OUTPUT_FOLDER_PLACE"],
                info=locale_data["LNG_OUTPUT_FOLDER_INFO"]
            )

    # --- 音声ファイル変換設定 ---
    with gr.Accordion(locale_data["LNG_AUDIO_PREPROCESS_SETTINGS"], open=True):
        prep_enable = gr.Checkbox(
            label=locale_data["LNG_ENABLE_PREPROCESS"], value=True,
            info=locale_data["LNG_ENABLE_PREPROCESS_INFO"]
        )
        with gr.Row():
            prep_duration = gr.Slider(minimum=1, maximum=30, step=1, value=5, label=locale_data["LNG_SEGMENT_DURATION_LABEL"], info=locale_data["LNG_SEGMENT_DURATION_INFO"])
            prep_sr = gr.Dropdown(choices=["16000", "22050", "24000", "32000", "44100", "48000"], value="16000", label=locale_data["LNG_OUTPUT_SAMPLERATE_LABEL"], info=locale_data["LNG_OUTPUT_SAMPLERATE_INFO"])
            prep_format = gr.Dropdown(choices=["wav", "flac"], value="wav", label=locale_data["LNG_OUTPUT_FORMAT_LABEL"], info=locale_data["LNG_OUTPUT_FORMAT_INFO"])
        with gr.Row():
            prep_silence_remove = gr.Checkbox(label=locale_data["LNG_SILENCE_REMOVAL_OPTIONS"], info=locale_data["LNG_SILENCE_REMOVAL_OPTIONS_INFO"], value=True)
            prep_silence_thresh = gr.Slider(minimum=-60, maximum=0, step=1, value=-40, label=locale_data["LNG_SILENCE_THRESHOLD_LABEL"], info=locale_data["LNG_SILENCE_THRESHOLD_INFO"])
            prep_min_silence = gr.Slider(minimum=100, maximum=5000, step=100, value=500, label=locale_data["LNG_MIN_SILENCE_DURATION_LABEL"], info=locale_data["LNG_MIN_SILENCE_DURATION_INFO"])

    # --- 基本トレーニング設定 ---
    with gr.Accordion(locale_data["LNG_BASIC_TRAINING"], open=True):
        with gr.Row():
            n_steps = gr.Number(value=default_config["n_steps"], label="n_steps", info=locale_data["LNG_N_STEPS_INFO"])
            batch_size = gr.Slider(minimum=1,maximum=64,step=1, value=default_config["batch_size"], label="Batch Size", info=locale_data["LNG_BATCH_SIZE_INFO"])
            num_workers = gr.Slider(minimum=0,maximum=64,step=1, value=default_config["num_workers"], label="Num Workers", info=locale_data["LNG_NUM_WORKERS_INFO"])
        with gr.Row():
            save_interval = gr.Number(value=default_config["save_interval"], label="Save Interval", info=locale_data["LNG_SAVE_INTERVAL_INFO"])
            evaluation_interval = gr.Number(value=default_config["evaluation_interval"], label="Evaluation Interval", info=locale_data["LNG_EVALUATION_INTERVAL_INFO"])

    # --- 詳細オプション ---
    with gr.Accordion(locale_data["LNG_ADVANCED_OPTIONS"], open=False):
        with gr.Accordion(locale_data["LNG_LEARNING_RATE_OPTIMIZER"], open=False):
            with gr.Row():
                learning_rate_g = gr.Number(label="Learning Rate G", value=default_config["learning_rate_g"])
                learning_rate_d = gr.Number(label="Learning Rate D", value=default_config["learning_rate_d"])
                learning_rate_decay = gr.Number(label="Learning Rate Decay", value=default_config["learning_rate_decay"])
                warmup_steps = gr.Number(label="Warmup Steps", value=default_config["warmup_steps"])
            with gr.Row():
                adam_betas_1 = gr.Number(label="Adam Betas 1", value=default_config["adam_betas"][0])
                adam_betas_2 = gr.Number(label="Adam Betas 2", value=default_config["adam_betas"][1])
                adam_eps = gr.Number(label="Adam Eps", value=default_config["adam_eps"])

        with gr.Accordion(locale_data["LNG_LOSS_WEIGHTS"], open=False):
            with gr.Row():
                grad_weight_loudness = gr.Number(label="Grad Weight Loudness", value=default_config["grad_weight_loudness"], step=0.1, precision=2)
                grad_weight_mel = gr.Number(label="Grad Weight Mel", value=default_config["grad_weight_mel"], step=0.1, precision=2)
                grad_weight_ap = gr.Number(label="Grad Weight AP", value=default_config["grad_weight_ap"], step=0.1, precision=2)
            with gr.Row():
                grad_weight_adv = gr.Number(label="Grad Weight Adv", value=default_config["grad_weight_adv"], step=0.1, precision=2)
                grad_weight_fm = gr.Number(label="Grad Weight FM", value=default_config["grad_weight_fm"], step=0.1, precision=2)
                grad_balancer_ema_decay = gr.Number(label="Grad Balancer EMA Decay", value=default_config["grad_balancer_ema_decay"])

        with gr.Accordion(locale_data["LNG_AUGMENTATION_OPTIONS"], open=False):
            with gr.Row():
                aug_formant_shift_prob = gr.Slider(minimum=0.0, maximum=1.0, label="Formant Shift Probability", value=default_config["augmentation_formant_shift_probability"])
                aug_formant_shift_min = gr.Number(label="Formant Shift Semitone Min", value=default_config["augmentation_formant_shift_semitone_min"])
                aug_formant_shift_max = gr.Number(label="Formant Shift Semitone Max", value=default_config["augmentation_formant_shift_semitone_max"])
            with gr.Row():
                aug_reverb_prob = gr.Slider(minimum=0.0, maximum=1.0, label="Reverb Probability", value=default_config["augmentation_reverb_probability"])
                aug_lpf_prob = gr.Slider(minimum=0.0, maximum=1.0, label="LPF Probability", value=default_config["augmentation_lpf_probability"])
            with gr.Row():
                aug_snr_candidates = gr.Textbox(label="SNR Candidates (comma-separated)", value=", ".join(map(str, default_config["augmentation_snr_candidates"])))
                aug_lpf_cutoff_candidates = gr.Textbox(label="LPF Cutoff Freq Candidates (comma-separated)", value=", ".join(map(str, default_config["augmentation_lpf_cutoff_freq_candidates"])))

        with gr.Accordion(locale_data["LNG_AUDIO_MODEL"], open=False):
            with gr.Row():
                in_sample_rate = gr.Number(label="In Sample Rate", value=default_config["in_sample_rate"], interactive=False, info=locale_data["LNG_IN_SAMPLE_RATE_INFO"])
                out_sample_rate = gr.Number(label="Out Sample Rate", value=default_config["out_sample_rate"], interactive=False, info=locale_data["LNG_OUT_SAMPLE_RATE_INFO"])
                wav_length = gr.Number(label="Wav Length", value=default_config["wav_length"])
                segment_length = gr.Number(label="Segment Length", value=default_config["segment_length"])
            with gr.Row():
                phone_noise_ratio = gr.Slider(minimum=0.0, maximum=1.0, label="Phone Noise Ratio", value=default_config["phone_noise_ratio"])
                floor_noise_level = gr.Number(label="Floor Noise Level", value=default_config["floor_noise_level"])
                vq_topk = gr.Number(label="VQ Top-K", value=default_config["vq_topk"], step=1)
                training_time_vq = gr.Dropdown(label="Training Time VQ", choices=["none", "self", "random"], value=default_config["training_time_vq"])
            with gr.Row():
                pitch_bins = gr.Number(label="Pitch Bins", value=default_config["pitch_bins"], interactive=False)
                hidden_channels = gr.Number(label="Hidden Channels", value=default_config["hidden_channels"])
                record_metrics = gr.Checkbox(label="Record Metrics", value=default_config["record_metrics"])

        with gr.Accordion(locale_data["LNG_FILE_PATHS"], open=False):
            with gr.Column():
                in_ir_wav_dir = gr.Textbox(label="In IR Wav Dir", value=default_config["in_ir_wav_dir"])
                in_noise_wav_dir = gr.Textbox(label="In Noise Wav Dir", value=default_config["in_noise_wav_dir"])
                in_test_wav_dir = gr.Textbox(label="In Test Wav Dir", value=default_config["in_test_wav_dir"])
                pretrained_file = gr.Textbox(label="Pretrained File", value=default_config["pretrained_file"])
                phone_extractor_file = gr.Textbox(label="Phone Extractor File", value=default_config["phone_extractor_file"])
                pitch_estimator_file = gr.Textbox(label="Pitch Estimator File", value=default_config["pitch_estimator_file"])

        with gr.Accordion(locale_data["LNG_PERFORMANCE_DEBUG"], open=False):
            with gr.Row():
                use_amp = gr.Checkbox(label="Use AMP", value=default_config["use_amp"])
                san = gr.Checkbox(label="SAN (Discriminator)", value=default_config["san"])
                profile = gr.Checkbox(label="Profile", value=default_config["profile"])
            with gr.Row():
                compile_convnext = gr.Checkbox(label="Compile ConvNext", value=default_config["compile_convnext"])
                compile_d4c = gr.Checkbox(label="Compile D4c", value=default_config["compile_d4c"])
                compile_discriminator = gr.Checkbox(label="Compile Discriminator", value=default_config["compile_discriminator"])

    # --- 実行ボタン・ステータス ---
    with gr.Row():
        add_to_queue_button = gr.Button(value=locale_data["LNG_ADD_TASK_BUTTON"], variant="primary")
        start_button = gr.Button(value=locale_data["LNG_START_TRAINING_BUTTON"], variant="primary", interactive=False)
        stop_button = gr.Button(value=locale_data["LNG_STOP_TRAINING_BUTTON"], variant="stop", interactive=False)

    queue_status_box = gr.Markdown(locale_data["LNG_TASK_MONITOR_EMPTY"])

    # --- イベントリスナー設定 ---
    prep_inputs = [prep_enable, prep_duration, prep_sr, prep_format, prep_silence_remove, prep_silence_thresh, prep_min_silence]

    all_inputs = [
        learning_rate_g, learning_rate_d, learning_rate_decay, adam_betas_1, adam_betas_2, adam_eps,
        batch_size, grad_weight_loudness, grad_weight_mel, grad_weight_ap, grad_weight_adv,
        grad_weight_fm, grad_balancer_ema_decay, use_amp, num_workers, n_steps, warmup_steps,
        evaluation_interval, save_interval,
        in_sample_rate, out_sample_rate, wav_length, segment_length, phone_noise_ratio, vq_topk,
        training_time_vq, floor_noise_level, record_metrics,
        aug_snr_candidates, aug_formant_shift_prob, aug_formant_shift_min, aug_formant_shift_max,
        aug_reverb_prob, aug_lpf_prob, aug_lpf_cutoff_candidates,
        phone_extractor_file, pitch_estimator_file, in_ir_wav_dir, in_noise_wav_dir,
        in_test_wav_dir, pretrained_file,
        pitch_bins, hidden_channels, san, compile_convnext, compile_d4c, compile_discriminator, profile
    ]

    add_to_queue_button.click(add_to_queue, inputs=[input_folder, output_folder] + prep_inputs + all_inputs, outputs=[queue_status_box, add_to_queue_button, start_button, stop_button])
    start_button.click(process_training_queue_generator, inputs=None, outputs=[start_button, add_to_queue_button, stop_button, queue_status_box])
    stop_button.click(stop_training, inputs=None, outputs=[add_to_queue_button, start_button, stop_button, queue_status_box])

demo.launch(inbrowser=True, share=True)
'''

with open("webui_colab.py", "w") as f:
    f.write(code)
print("webui_colab.py を出力しました。")

In [ ]:
# @title 3. WebUIの起動
!python webui_colab.py